# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a membrane defined by a mesh

In [ ]:
from bioexplorer import BioExplorer, MeshBasedMembrane, Vector3, Quaternion
be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
obj_folder = resource_folder + 'obj/'
membrane_folder = pdb_folder + 'membrane/'

### Configuration

In [ ]:
mesh_source= obj_folder + 'suzanne.obj'
scale = Vector3(2.5, 2.5, 2.5)

protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_atom_radius_multiplier = 1.0

## Camera position

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.114, 0.341, 0.020, 0.932],
    position=[368.152, 107.242, 367.312],
    target=[50.364, -6.824, -5.159]
)

### Membrane

In [ ]:
def add_membrane(
        frame, random_seed=1, add_receptors=False,
        position=Vector3(0.0, 0.0, 0.0), 
        rotation=Quaternion(1.0, 0.0, 0.0, 0.0)):
    
    clip_planes = [[-1.0, 0.0, 0.0, 50.0]]
    params1 = [0, 0, 0.0, frame * 2 + 1, 1.0]
    params2 = [0, 0, 0.0, frame * 3 + 2, 0.5]

    '''Membrane'''
    name = 'Suzanne'
    protein_sources = [
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ]
    
    mesh_based_membrane = MeshBasedMembrane(
        mesh_source=mesh_source, 
        protein_sources=protein_sources, 
        representation=protein_representation,
        atom_radius_multiplier=protein_atom_radius_multiplier,
        density=5.0, random_seed=random_seed, 
        assembly_params=params1,
        surface_variable_offset=0.0)

    status = be.add_mesh_based_membrane(
        name=name, mesh_based_membrane=mesh_based_membrane,
        position=position, scale=scale, rotation=rotation,
        clipping_planes=clip_planes
    )

    for i in range(len(protein_sources)):
        status = be.set_protein_color_scheme(
            assembly_name=name, name=be.NAME_MEMBRANE + '_' + str(i),
            color_scheme=be.COLOR_SCHEME_CHAINS,
            palette_name='Set3', palette_size=5)
        
    if add_receptors:
        '''Receptors'''
        name = be.NAME_RECEPTOR
        protein_sources = [pdb_folder + '6m18.pdb']

        mesh_based_membrane = MeshBasedMembrane(
            mesh_source=mesh_source, protein_sources=protein_sources,
            representation=protein_representation,
            density=0.05, atom_radius_multiplier=protein_atom_radius_multiplier,
            assembly_params=params2, surface_fixed_offset=3,
            random_seed=random_seed)

        status = be.add_mesh_based_membrane(
            name, mesh_based_membrane,
            position=position, scale=scale, rotation=rotation,
            clipping_planes=clip_planes
        )

        status = be.set_protein_color_scheme(
            assembly_name=name, name=be.NAME_MEMBRANE + '_0',
            color_scheme=be.COLOR_SCHEME_CHAINS,
            palette_name='OrRd', palette_size=5)        

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Animation

In [ ]:
from mediamaker import MovieMaker
from IPython.display import clear_output

mm = MovieMaker(be)
output_folder = '/tmp'
pos = Vector3(0.0, 0.0, 0.0)
rot_start = Quaternion(1.0, 0.0, 0.0, 0.0)
rot_end = Quaternion(0.0, 0.0, 1.0, 0.0)

nb_frames = 36

for frame in range(0, nb_frames):
    clear_output()
    be.reset()
    print('Frame %d' % frame)
    rot = Quaternion.slerp(rot_start, rot_end, float(frame) / nb_frames)
    add_membrane(
        frame=frame, add_receptors=True,
        position=pos, rotation=rot)
    
    '''Snapshot'''
    mm.create_snapshot(
        size=[512, 512], samples_per_pixel=16,
        path=output_folder,
        base_name='%05d' % frame)